In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [2]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [3]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [4]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'):
        if not 'Primera' in info.get_text(): #Exclude 'Primera Publicacion'
            if not 'Contrato' in info.get_text():
                anuncio_item = info.get_text()
                anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
                anuncio_list.append(anuncio_item.replace('\n',''))  #Get items without \n
                #anuncio_list.append(info.get_text().replace('\n',''))
    return anuncio_list

In [5]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [6]:
def get_name_empresa(result_page):
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    return Enterprise_information.find('strong').get_text()

In [7]:
def clean_text(text):
    import unidecode
    text = text.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [8]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [10]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
description_list = get_job_description(result_page)
empresa_name = get_name_empresa(result_page)
output = unify_lists([empresa_name, anuncio_list, description_list])

In [11]:
output

['NOSSAMODA SRL',
 '695472',
 'Santa Cruz de la Sierra, Bolivia',
 'Contabilidad y Economía ',
 'De Bs. 3.000 a Bs. 3.999 ',
 '28 Enero 2019',
 '27 Febrero 2019',
 'analista de tesoreria formacion titulado en auditoria financiera contaduria publica contaduria general o ramas afines experiencia y conocimientos  tres  anos de experiencia laboral en cargos de igual responsabilidad  elaboracion de conciliaciones bancarias y arqueos de caja elaboracion de flujos de caja gestion de pagos a proveedores locales y extranjeros seguimiento de cuentas pendientes de cobro y depositos de cobranzas del dia  preparacion de estados de cuentas por cobrar y pagar y control de emision de cheques registro contable de pagos a proveedores sueldos finiquitos etc  registro contable de cobros a clientes depositos transferencias etc  conocimiento en sistema contable sai   solidos manejo de windows office excel powerpoint outlook y word competencias  responsabilidad  organizacion  atencion a los detalles las pers

In [12]:
def trabajopolis(job_description):
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    for job in job_description:
        #url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job + "&page=1&listings_per_page=1000"
        url = "https://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D=cajero&page=1&listings_per_page=10"

        response = requests.get(url)
        if response.status_code == 200:
            print("The request response cycle was successful")
            results_page = BeautifulSoup(response.content,'lxml')
            #trabajo_list = list()
            trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
            for anuncio in trabajos:
                trabajo_link = anuncio.find('a').get('href')
                trabajo_name = anuncio.find('b').get_text()
                print(trabajo_link)
             #   trabajo_list.append((trabajo_name,trabajo_link))
            #for trabajo in trabajo_list:
                output = get_job_info(trabajo_link)
                trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
        else:
            print("The request response cycle work didn't work")
        time.sleep(10) 
    return trabajos_list

In [13]:
def trabajopolis2():
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.trabajopolis.bo/search-results-jobs/?searchId=1546400657&action=search&page=1&listings_per_page=1500"
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_name)
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [14]:
Results = trabajopolis2()

The request response cycle was successful
Ejecutivo(a) de Ventas
Chofer
Jefe de Importaciones
Analista de Calidad de Software - La Paz
Coordinador(a) Trade Marketing - Santa Cruz
Secretaria(o) Ejecutiva(o) - La Paz
Secretaria(o) Ejecutiva(o) - Santa Cruz
Activador(a) Freelancer - La Paz
Activador(a) Freelancer - Cochabamba
Asistente de Gerencia
Jefe de Servicio Técnico
Pasantía en Gestión de Recursos Humanos - Cochabamba
Gerente de Ventas de Repuestos
Community Manager
Ejecutivos(as) de Ventas - Santa Cruz
Vendedor(a) de Provincias - La Paz
Recepcionista
Ejecutivo(a) de Ventas - La Paz
Contador(a) - La Paz
Oficial de Créditos - Santa Cruz
Jefe de Agencia - Santa Cruz
Inspector(a) Zoosanitario(a)
Auxiliar de Compras
Auxiliar Contable
Técnico en Mantenimiento Mecánico
Responsable Regional de Siniestros - La Paz
Responsable Nacional de Siniestros Seguros Personales
Programador(a) ODOO ERP – Python
Auxiliar Contable - Cochabamba
Jefe Nacional de Siniestros - La Paz
Auxiliar de RR.HH.
Jefe 

Pasante de la Carrera de Psicología
Ejecutivo de Ventas Camiones & Buses - Santa Cruz
Ejecutivo(a) de Ventas
Oficial de Ventas
Asistente de Logística y Almacenes
Recepcionista
Team Leader de Ventas Hogar - Yacuiba
Liquidador(a)
Analista de Reclutamiento y Selección - Santa Cruz
Pasante de Planificación
Técnico de Servicio, Soporte y Mantenimiento Equipos IOS
Asistente Contable (Contrato Temporal)
Encargado(a) de Redes Sociales
Jefe Administrativo de Maestranza
Vendedor(a) Ruta -Santa Cruz
Ejecutivo(a) de Ventas (Nivel Corporativo)
Ejecutivos(as) de Ventas
Jefe Regional La Paz y El Alto
Diseñador(a) Gráfico(a)
Responsable de Toma de Inventarios / Consultor(a) por Producto
Programa de Desarrollo Comercial - Cochabamba
Asistente de Innovación y Desarrollo - Santa Cruz
Auxiliar Contable
Pasante de Comercial
Analista de Cuentas Especiales - Potosí
Ejecutivo(a) Comercial
Customer Service Operator - Potosí
Coordinador(a) Administrativo(a) - Oruro
Ejecutivo(a) de Ventas Hogar - Potosí
Ejecutiv

Vendedor(a) de Provincia
Vendedor(a)
Ejecutivo(a) de Ventas AUDI - La Paz
Auxiliar de Almacén - Uyuni
Analista de Seguridad Informática
Ventas Showroom
Ejecutivo(a) de Ventas - Santa Cruz
Jefe Nacional de Categoría - Lácteos
Jefe Nacional de Categoría - Bebidas
Jefe Nacional de Categoría - Helados
Ingeniero(a) de Ventas - Productor Eléctricos
Controller Administrativo(a) y de Sistemas - Santa Cruz
Instructor(a) Técnico(a) Automotriz
Auxiliar de Compras - Santa Cruz
Especialista E-Commerce
Asistente de Oficina
Ejecutivo(a) de Ventas Línea Baterías - La Paz
Integrador(a) Tecnológico(a) de Productos y Soluciones
Insighter de Negocio
Auxiliar de Spa
Insighter de Mercado
Supervisor(a) de Ventas - Santa Cruz
Administrativo(a) de Inteligencia de Clientes y Datos - Santa Cruz
Maestro Electromecánico - Santa Cruz
Analista Tributario(a)
Auditor(a) Interno(a)
Programador(a)
Supervisor(a) Contable Tributario(a) - Santa Cruz
Auxiliar de Tienda (Color-Shop) - El Alto
Supervisor(a) de Operaciones - C

Supervisor(a) de Distribución Nacional - Santa Cruz
Arquitecto(a) Júnior
Encargado(a) de Almacén de Campo
Auxiliar Administrativo(a) de Campo
Asistente Administrativo(a)
Coordinador(a) de Reclutamiento y Selección - Cochabamba
Coordinador(a) de Tecnologías de la Información
Jefe de Operaciones - Santa Cruz
Director(a) de Centro de Idiomas
Analista Comercial
Ejecutivo(a) de Atención al Cliente y Asociado - Santa Cruz
Coordinador(a) de Operaciones - La Paz
Fisioterapeuta
Psicólogo(a)
Ejecutivo(a) de Cobranza - Riberalta
Auxiliar Contable
Mecánico Industrial
Auxiliar Contable - La Paz
Analista de Riesgo de Crédito - La Paz
Experto(a) en Facebook Ads - Santa Cruz
Coordinador(a) CSSMA (División Catering)
Programador(a)
Encargado(a) de Supermercado
Supervisor(a) de Ventas y Cobranzas - Canal Mercados
Arquitecto(a) - La Paz
Analista Financiero(a) - Warnes
Asesor(a) Comercial - Santa Cruz
Gerente de Ventas - Línea Bebidas
Analista de Admisión Crediticia - Santa Cruz
Jefe de Ventas
Docente de G

In [32]:
len(Results)

885

In [38]:
import csv
# Open the file in the mode append and the encoding utf-8
outfile = open("/Users/Roberto/Documents/Python Scripts/Machine Learning Projects/Labor Market/TRABAJOPOLIS_FEB2019.csv", 'a',newline='', encoding = 'utf-8')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        # Transform the list shape:
        if Results[i][0] is not None:
            
            for cell in Results[i][0]:
                csvRow.append(cell)
                
            csvRow.append(Results[i][1])
            csvRow.append(Results[i][2])

            writer.writerow(csvRow)
finally:
    outfile.close() 

In [34]:
Results[400]

(['Compañía de Alimentos Ltda. (Delizia)',
  '727244',
  'La Paz, Bolivia',
  'Marketing y Ventas ',
  'De Bs. 4.000 a Bs. 4.999 ',
  '04 Marzo 2019',
  '03 Abril 2019',
  'estamos en busqueda de excelencia profesional expertise ejemplar compromiso con la formalidad y pasion por la efectividad valores plasmados en un lider solido y etico para asumir en compania de alimentos ltda  el cargo de coordinador de investigacion de mercadosede la paz objetivo del cargo esta oportunidad laboral es ideal para individuos que recientemente hayan culminado sus estudios universitarios o se encuentren terminando sus ultimos proyectos de grado y se mantengan interesados en tener una primera experiencia laboral que los sumerja a la disciplina del marketing dentro de la industria alimenticia e industrias afines individuos con una amplia capacidad de ejecucion facilidades para trabajo de campo alto nivel de organizacion y voluntad para trabajar bajo presion son candidatos ideales requisitos ser profesiona

In [35]:
Results[0]

(['Placacenter S.R.L.',
  '733836',
  'Santa Cruz de la Sierra, Bolivia',
  'Atención al Cliente,Marketing y Ventas ',
  'De Bs. 3.000 a Bs. 3.999 ',
  '16 Marzo 2019',
  '15 Abril 2019',
  'empresa lider en importacion y comercializacion de productos para la construccion necesita contratar ejecutivo de ventas objetivo del cargocontribuir con el logro de las metas de ventas mensuales y proyecciones anuales de la empresa brindando una calidad de servicio al cliente que marque la diferencia en relacion a las empresas del mercado requisitos profesional o estudiante en ingenieria comercial marketing o ramas afines amplia experiencia en ventas yo cargos similares capacidad de trabajo en equipo y bajo presion dinamico emprendedor y proactivo facilidad de palabra habilidad de negociacion y poder de convencimiento solidos conocimientos de microsoft office e internet disponibilidad inmediata ofrecemos salario fijo mas comisiones en base a resultados de ventas estabilidad laboral las personas in

In [30]:
Results = trabajopolis(['cajero'])

The request response cycle was successful
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/716635/Cajero(a).html?searchId=1549376419.06&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/716360/Auxiliar-Administrativo(a)-Contable.html?searchId=1549376419.06&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/715651/Cajero(a)---El-Alto.html?searchId=1549376419.06&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/715416/Cajero(a)---Santa-Cruz.html?searchId=1549376419.06&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/714946/Cajero(a)---Tarija.html?searchId=1549376419.06&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/712690/Cajero(a)---La-Paz.html?searchId=1549376419.06&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/712230/Cajero(a)---Santa-Cruz.html?searchId=1549376419.06&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empl

In [60]:
hola = Results[1000:]

In [65]:
for i in range(len(hola)-1):
    csvRow = []
        
    if Results[i][0] is not None:
            
        for cell in hola[i][0]:
            csvRow.append(cell)
                
        csvRow.append(hola[i][1])
        csvRow.append(hola[i][2])

print(csvRow)

['Colser Ltda.', '700615', 'La Paz, Bolivia', 'Administración y Oficina,Atención al Cliente ', 'No Declarado ', '07 Enero 2019', '06 Febrero 2019', 'we are an international software development company with a world class bpm product we are looking for a talented professional with the following profile to join our team customer success agent description of the positionthe customer success agent will work alongside the customer success team to ensure our customers increase in product satisfaction adoption  renewal throughout our current global client network we are looking for a young talented dynamic and well organized professional with strong interpersonal skills and learning abilities in the it environment  only applicants with excellent spoken english will be considered administrative responsibilities keep good records in the crm of all account renewal activities setup agenda and activities for customer success managers  assist on client renewal process reports to csms qualifications

In [53]:
csvrow = []
for cell in Results[i][0]:
    #print(cell)
    csvrow.append(cell)
csvrow.append(Results[i][1])
csvrow.append(Results[i][2])
print(csvrow)

['Guayabochi srl', '699870', 'Santa Cruz de la Sierra, Bolivia', 'Administración y Oficina ', 'No Declarado ', '07 Enero 2019', '06 Febrero 2019', 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo', 'Mensajero(a)', 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/699870/Mensajero(a).html?searchId=1546400657&page=1']


In [43]:
Results[i][0]

['Guayabochi srl',
 '699870',
 'Santa Cruz de la Sierra, Bolivia',
 'Administración y Oficina ',
 'No Declarado ',
 '07 Enero 2019',
 '06 Febrero 2019',
 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo']

In [44]:
for cell in Results[i][0]:
    print(cell)

Guayabochi srl
699870
Santa Cruz de la Sierra, Bolivia
Administración y Oficina 
No Declarado 
07 Enero 2019
06 Febrero 2019
empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo


In [45]:
csvRow.append(cell)